## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
import import_ipynb

In [3]:
import os 

In [4]:
os.getcwd()

'C:\\Users\\yuuna\\Weather_Analysis\\World_Weather_Analysis\\Vacation_Search'

In [5]:
import sys 

In [6]:
sys.path.append("C:\\Users\\yuuna\\Weather_Analysis\\World_Weather_Analysis")

In [7]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [8]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("C:\\Users\\yuuna\\Weather_Analysis\\World_Weather_Analysis\\Weather_Database\\WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Juba,SS,4.85,31.58,84.61,55,63,5.35,broken clouds
1,1,Hithadhoo,MV,-0.60,73.08,83.01,77,100,14.47,overcast clouds
2,2,Grand-Santi,GF,4.25,-54.38,71.69,96,35,2.68,scattered clouds
3,3,Osmena,PH,10.52,119.93,85.21,74,7,9.95,clear sky
4,4,Bredasdorp,ZA,-34.53,20.04,68.00,56,57,3.36,broken clouds


In [10]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 100


In [11]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Juba,SS,4.85,31.58,84.61,55,63,5.35,broken clouds
1,1,Hithadhoo,MV,-0.60,73.08,83.01,77,100,14.47,overcast clouds
2,2,Grand-Santi,GF,4.25,-54.38,71.69,96,35,2.68,scattered clouds
3,3,Osmena,PH,10.52,119.93,85.21,74,7,9.95,clear sky
10,10,Sambava,MG,-14.27,50.17,80.73,60,2,15.17,clear sky
14,14,Bullhead City,US,35.15,-114.57,73.40,20,1,1.77,clear sky
18,18,Hilo,US,19.73,-155.09,75.20,83,90,4.70,overcast clouds
19,19,Wattegama,LK,6.80,81.48,93.20,56,40,11.41,scattered clouds
24,24,Sur,OM,22.57,59.53,87.26,35,13,20.56,few clouds
27,27,Omboue,GA,-1.57,9.26,80.89,79,18,9.40,few clouds


In [12]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                296
City                   296
Country                292
Lat                    296
Lng                    296
Max Temp               296
Humidity               296
Cloudiness             296
Wind Speed             296
Current Description    296
dtype: int64

In [13]:
print(preferred_cities_df[preferred_cities_df['Country'].isnull()])

     City_ID       City Country    Lat    Lng  Max Temp  Humidity  Cloudiness  \
82        82      Opuwo     NaN -18.06  13.84     92.93        11           0   
153      153   Luderitz     NaN -26.65  15.16     71.20        36           0   
245      245    Gobabis     NaN -22.45  18.97     89.06        11           5   
404      404  Okakarara     NaN -20.58  17.43     89.15        11           0   

     Wind Speed Current Description  
82        14.97           clear sky  
153       19.84           clear sky  
245       12.37           clear sky  
404        9.04           clear sky  


In [14]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
new_preferred_cities_df=preferred_cities_df.dropna()
new_preferred_cities_df.count()

City_ID                292
City                   292
Country                292
Lat                    292
Lng                    292
Max Temp               292
Humidity               292
Cloudiness             292
Wind Speed             292
Current Description    292
dtype: int64

In [15]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = new_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Juba,SS,84.61,broken clouds,4.85,31.58,
1,Hithadhoo,MV,83.01,overcast clouds,-0.60,73.08,
2,Grand-Santi,GF,71.69,scattered clouds,4.25,-54.38,
3,Osmena,PH,85.21,clear sky,10.52,119.93,
10,Sambava,MG,80.73,clear sky,-14.27,50.17,
14,Bullhead City,US,73.40,clear sky,35.15,-114.57,
18,Hilo,US,75.20,overcast clouds,19.73,-155.09,
19,Wattegama,LK,93.20,scattered clouds,6.80,81.48,
24,Sur,OM,87.26,few clouds,22.57,59.53,
27,Omboue,GA,80.89,few clouds,-1.57,9.26,


In [16]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [17]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].isna().sum()

0

In [23]:
#drop the rows where there is no hotel name.
hotel_df=hotel_df.loc[(hotel_df["Hotel Name"]!="")]

In [24]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Juba,SS,84.61,broken clouds,4.85,31.58,ROYAL PALACE HOTEL - JUBA
1,Hithadhoo,MV,83.01,overcast clouds,-0.60,73.08,Scoop Guest House
2,Grand-Santi,GF,71.69,scattered clouds,4.25,-54.38,Hotel La Villa Couacou
3,Osmena,PH,85.21,clear sky,10.52,119.93,Arjemarkyza Lodge
10,Sambava,MG,80.73,clear sky,-14.27,50.17,Melrose
...,...,...,...,...,...,...,...
673,Manokwari,ID,80.37,light rain,-0.87,134.08,Swiss Belhotel Manokwari
675,Mahanoro,MG,77.29,light rain,-19.90,48.80,Hôtel Mon Désir
677,Bulawayo,ZW,88.25,clear sky,-20.15,28.58,4 On Housman Backpackers
681,Presidente Dutra,BR,73.11,few clouds,-5.29,-44.49,Pousada Planalto


In [25]:
# 8a. Create the output File (CSV)
output_data_file = "C:\\Users\\yuuna\\Weather_Analysis\\World_Weather_Analysis\\Vacation_Search\\WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [26]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [27]:
# 11a. Add a marker layer for each city to the map. 
# 11b. Display the figure

# Get the latitude and longitude.
locations = hotel_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = hotel_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
#add a marker
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))